# 4. CVaR MDP optimizations

### <font color='red'> UNDER CONSTRUCTION !!!:</font>

There are two types of update for the available CVaR-MDP.

1. Speed: There are three possible area where speed could be optimized in CVaR MDP. 

    1. Calculation: Obtain CVaR from distribution and recover distribution from the CVaR, are inverse function. Instead of disposing the original distribution and recover from CVaR, we could directly construct the optimistic distribution directly.
    
    2. Sort: When constructing joint distribution from conditionals, we could use merge sort and assumed that the conditional distribution are already sorted. This modification will lead us from $O(SM log(SM))$ to $O(SM log(S) )$.
    
    3. Discretization: In order to save computation, we could compute the optimal discretization of CVaR at only P$\Sigma$.
    

2. Accuracy: Note that the original CVaR MDP suffers from accurate calculation of value function shown above.


In [26]:
df = CSV.read("C:/GITHUB/rmdp-jl-2/data/riverswim.csv", DataFrame)
# The document uses "zero index" so we need to change to "one index for julia"
df[:,["idstatefrom","idaction","idstateto","idoutcome"]] = df[:,["idstatefrom","idaction","idstateto","idoutcome"]] .+ 1
# Convert a data frame to MDP.
function df2MMDP(df;γ = 0.90)
    S = unique([df.idstatefrom;df.idstateto])
    A = unique(df.idaction)
    M = unique(df.idoutcome)
    lSl = length(S)
    lAl = length(A)
    lMl = length(M)
    P = zeros((lMl,lSl,lAl,lSl))
    R = zeros((lMl,lSl,lAl,lSl))
    for m in M
        dftemp = filter(x -> x.idoutcome == m ,df)
        tempmdp = df2MDP(dftemp;γ = γ)
        P[m,:,:,:] = tempmdp.P
        R[m,:,:,:] = tempmdp.R
    end
    return (S=S,A=A,P=P,R=R,lSl=lSl,lAl=lAl,γ=γ)
end

function MMDP2MDP(MMDP)
    return(S=MMDP.S,A=MMDP.A,P=mean(MMDP.P,dims=1)[1,:,:,:],R=mean(MMDP.R,dims=1)[1,:,:,:],lSl=MMDP.lSl,lAl=MMDP.lAl,γ=MMDP.γ)
end

mdp = MMDP2MDP(df2MMDP(df))
    
M = 101
T = 100
sol = CVaR_MDP(mdp;T=T,M=M)
@elapsed CVaR_MDP(mdp;T=T,M=M)

1.3411899

### Accuracy
When we are interested in computing accurate CVaR values in at least finite horizon problem, we will need to understand what causes the error in the first place. 

1. Discretization


2. Over-estimation
How to remove over estimation, one method to do so is to allow more information to be passed through at each iteration.

    1. The question is what are the crucial information to store and what information could be dispose?
    
##### Look ahead (CVaR MDP)
1. Initialize $V_{T+1}(S,\alpha s) \gets 0$.
2. For $t \in (T:1)$:
    1. $Q_t(s, \cdot ,a) \gets ~R(s,a,S') + \gamma V_{t+1}(S',~\cdot~)~\qquad, \forall s,a$
    2. Compute $\hat{Q}_t(s,\alpha,a) \gets \text{CVaR}_\alpha(~Q_t(s,\alpha,a)~)\qquad, \forall s,\alpha,a$
    3. If (optimisic($\alpha$)):
    4. $\hat{V}_t(s,\alpha) \gets \max_{a}\{Q_t(s,\alpha,a) \}\qquad, \forall s,\alpha$
    5. $\pi_t(s,\alpha) \gets \arg \max_{a}\{Q_t(s,\alpha,a) \}\qquad, \forall s,\alpha$
    6. $V_t(s,~\alpha~) \gets Q_t(s, ~\alpha~ , \pi_t(s,\alpha) )\qquad, \forall s,\alpha$
3. Return $\pi,\hat{Q},V$